In [1]:
import torch
import torch.nn as nn
import os
import pandas as pd
import utm
import random
import Transformer as tr
import preprocess as pr
import simpleTrajVisualizer as vis
import random
from sklearn.model_selection import train_test_split
from sklearn.externals import joblib

In [ ]:
trajectory_data_file_path = ""  ## path to trajectory dataset
trained_model_file_path = ""    ## path to save trained model 
scalar_file_path = ""           ## path to save fitted scalar (leave empty if none)

number_of_bins = 30*30          ## number of bins
lr = 0.1                        ## learning rate 

In [2]:
traj_data = pd.read_csv(trajectory_data_file_path)

In [4]:
train_data_inputs, test_data_inputs, train_data_targets, test_data_targets, train_indx, test_indx = pr.preprocess_dataset(traj_data, max = 100, max_len = 50, input_len = 20, False, "")

In [5]:
if __name__ == "__main__":
  device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
  src_pad_idx = -999
  trg_pad_idx = -999
  model = tr.Transformer(number_of_bins, number_of_bins, src_pad_idx, trg_pad_idx, device=device, dropout=0.15).to(device)

10100 256
10100 256


In [6]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1.0, gamma=0.95)

import time
import math
def train():
    model.train() # Turn on the train mode
    total_loss = 0.
    start_time = time.time()
    for batch, i in enumerate(range(0, train_data_inputs.size(0), 1)):
      data = train_data_inputs[i,:,:]
      targets = train_data_targets[i,:,:]
      optimizer.zero_grad()
      output = model(data, targets[:, :])
      loss = criterion(output[0,:,:], targets[0,:])
      loss.backward()
      torch.nn.utils.clip_grad_norm_(model.parameters(), 0.5)
      optimizer.step()
      total_loss += loss.item()
        
    cur_loss = total_loss / train_data_inputs.size(0)
    print('lr {:02.2f} | '
                'loss {:5.2f} | ppl {:8.2f}'.format(
                  scheduler.get_lr()[0],
                  cur_loss, math.exp(cur_loss)))

def evaluate(eval_model, data_source_inputs, data_source_targets):
    eval_model.eval() # Turn on the evaluation mode
    total_loss = 0.

    with torch.no_grad():
        for i in range(0, data_source_inputs.size(0), 1):
            # data, targets = get_batch(data_source, i)
            data = data_source_inputs[i,:,:]
            targets = data_source_targets[i,:,:]
            output = eval_model(data, targets[:, :])
            output_flat = output
            total_loss += 1 * criterion(output[0,:,:], targets[0,:]).item()

    print("loss: ", total_loss / (data_source_inputs.size(0)))
    return total_loss / (data_source_inputs.size(0))


def get_predictions(eval_model, data_source_inputs, data_source_targets):
    eval_model.eval() # Turn on the evaluation mode
    preds_list = []
    target_list = []
    inputs_list = []
    with torch.no_grad():
        for i in range(0, data_source_inputs.size(0), 1):
            data = data_source_inputs[i,:,:]
            targets = data_source_targets[i,:,:]
            output = eval_model(data, targets[:, :])
            preds_list.append(torch.argmax(output[0,:,:], dim=1).tolist())
            target_list.append(targets[0,:].tolist())
            inputs_list.append(data[0,:].tolist())

    return inputs_list, target_list, preds_list

def displacement_error(x_diff, y_diff):
    squared_dist = np.linalg.norm((x_diff,y_diff))**2
    return squared_dist

def calculate_performance_metrics(targets, preds):
  x_diff = []
  y_diff = []
  
  for indx in range(0, len(targets)):
    yt = targets[indx] % 100
    xt = targets[indx] / 100

    yp = preds[indx] % 100
    xp = preds[indx] / 100

    tar_utm = utm.from_latlon(yt, xt)
    pred_utm = utm.from_latlon(yp, xp)

    dist_x = tar_utm[0] - pred_utm[0]
    dist_y = tar_utm[1] - pred_utm[1]

    x_diff.append(dist_x)
    y_diff.append(dist_y)

  
  dict_pd = {'X_DIFF': x_diff, 'Y_DIFF': y_diff} 
  output_df = pd.DataFrame(dict_pd)
  output_df[['X_DIFF', 'Y_DIFF']] = scaler.inverse_transform(output_df[['X_DIFF', 'Y_DIFF']])
  output_df['DIST'] = output_df.apply(lambda row: displacement_error(row['X_DIFF'], row['Y_DIFF']), axis=1)  
  ADE = output_df['DIST'].mean()
  FDE = output_df.tail(1)['DIST'].item()
  
  return (ADE, FDE) ## Average Displacement Error ADE, Final Displacement Error (FDE)

In [7]:
best_val_loss = float("inf")
epochs = 100 # The number of epochs
best_model = None
early_stopping = 0

for epoch in range(1, epochs + 1):
  epoch_start_time = time.time()
  train()
  val_loss = evaluate(model, test_data_inputs, test_data_targets)
  print('-' * 89)
  print('| end of epoch {:3d} | time: {:5.2f}s | valid loss {:5.2f} | '
        'valid ppl {:8.2f}'.format(epoch, (time.time() - epoch_start_time),
                                    val_loss, math.exp(val_loss)))
  print('-' * 89)

  early_stopping = early_stopping + 1

  if val_loss < best_val_loss:
    early_stopping = 0
    best_val_loss = val_loss
    best_model = model

  if early_stopping > 3:
    break

  scheduler.step()

KeyboardInterrupt: 

In [ ]:
evaluate(model, test_data_inputs, test_data_targets)

In [ ]:
torch.save(model, trained_model_file_path) ## save trained model

In [ ]:
joblib.dump(scaler, scalar_file_path)  ## save fitted scaler

In [ ]:
vis.vis_trajectory_scatter(best_model, test_data_inputs, test_data_targets) ## simple visualization of trajectories